<a href="https://colab.research.google.com/github/Nachocheeze18/atlas-machine_learning/blob/main/reinforcement_learning%5Cdeep_q_learning%5CMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf

In [2]:
from platform import version
!pip install keras-rl2 --quiet
!pip install gym[atari] --quiet
!pip install atari-py --quiet
!pip install gym[accept-rom-license] --quiet
!pip install tensorflow==2.11 --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!python -m atari_py.import_roms /content/drive/MyDrive/dqn/roms/

In [5]:
import gym
from gym.envs.registration import register

In [6]:
register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment Breakout-v4
  logger.warn(f"Overriding environment {spec.id}")


In [7]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from gym.wrappers import RecordVideo
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.callbacks import Callback
import json

class CustomModelCheckpoint(Callback):
    def __init__(self, filepath, monitor='val_loss', verbose=0, save_best_only=False,
                 save_weights_only=False, mode='auto', period=1):
        super(CustomModelCheckpoint, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.verbose = verbose
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.period = period
        self.epochs_since_last_save = 0
        self.best = np.Inf if 'loss' in monitor else -np.Inf
        self.train_logs = {'episode_reward': []}

    def on_train_begin(self, logs=None):
        self.train_logs = {'episode_reward': []}

    def on_step_end(self, step, logs=None):
        episode_reward = logs['episode_reward'] if 'episode_reward' in logs else None
        if episode_reward is not None:
            self.train_logs['episode_reward'].append(episode_reward)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    pass
                else:
                    if self.monitor_op(current, self.best):
                        self.best = current
                        if self.verbose > 0:
                            print(f'\nEpoch {epoch + 1}: {self.monitor} improved from '
                                  f'{self.best:.5f} to {current:.5f}, saving model to {filepath}')
                        if self.save_weights_only:
                            self.model.save_weights(filepath, overwrite=True)
                        else:
                            self.model.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print(f'\nEpoch {epoch + 1}: {self.monitor} did not improve')

            else:
                if self.verbose > 0:
                    print(f'\nEpoch {epoch + 1}: saving model to {filepath}')
                if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                else:
                    self.model.save(filepath, overwrite=True)

# Environment setup
env = gym.make('Breakout-v4')
env = RecordVideo(env, '/content/drive/MyDrive/dqn/deepqbreakout.mp4', episode_trigger=lambda episode: True)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Build a simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Configure and compile our agent
memory = SequentialMemory(limit=1000000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1., value_min=.1,
                              value_test=.05, nb_steps=5000)

agent = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=1e-2, policy=policy)
agent.compile(Adam(lr=1e-3), metrics=['mae'])

# Define the custom callback for logging
checkpoint = CustomModelCheckpoint('policy.h5', monitor='mae', verbose=1,
                                   save_best_only=True, mode='min', period=1)

# Train the agent
agent.fit(env, nb_steps=50000, visualize=False, verbose=2, callbacks=[checkpoint])

# Save the training logs to a JSON file
with open('training_log.json', 'w') as f:
    json.dump(checkpoint.train_logs, f)


/usr/local/lib/python3.10/dist-packages/gym/envs/atari/environment.py:68: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/dqn/deepqbreakout.mp4 folder (try specifying a different `video_

Training for 50000 steps ...


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


   498/50000: episode: 1, duration: 3.719s, episode steps: 498, steps per second: 134, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.600 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment Breakout-v4 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


...

 28566/50000: episode: 37, duration: 297.432s, episode steps: 876, steps per second:   3, episode reward:  3.000, mean reward:  0.003 [ 0.000,  1.000], mean action: 1.979 [0.000, 3.000],  loss: 0.000595, mae: 0.078177, mean_q: 0.105908, mean_eps: 0.100000


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment Breakout-v4 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


 29265/50000: episode: 38, duration: 239.860s, episode steps: 699, steps per second:   3, episode reward:  1.000, mean reward:  0.001 [ 0.000,  1.000], mean action: 1.555 [0.000, 3.000],  loss: 0.000544, mae: 0.083724, mean_q: 0.112760, mean_eps: 0.100000


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment Breakout-v4 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


done, took 6850.149 seconds
